# Experiments with Taiwan

In [1]:
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.inspection import PartialDependenceDisplay
from lime import lime_tabular
import shap
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier
from cfmining.visualization import buildTable, PlotCounterfactuals
from cfmining.mip_builder import RecourseBuilder
from cfmining.action_set import ActionSet
from aif360.algorithms.preprocessing import Reweighing
from fairlearn.postprocessing import ThresholdOptimizer
from fairgbm import FairGBMClassifier
from aif360.datasets import BinaryLabelDataset
from sklego.linear_model import EqualOpportunityClassifier, DemographicParityClassifier
import plotly.express as px
import plotly.graph_objects as go
import joblib


import credit_pipeline.data_exploration as dex
from credit_pipeline.training import *


%load_ext autoreload
%autoreload 2

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Loading and Cleaning

In [2]:
seed_number = 0

In [3]:
path = "../data/Taiwan/"
df = dex.read_csv_encoded(path, 'Taiwan.csv')
df.columns = df.iloc[0, :].tolist()
df = df.iloc[1:, :]
df = df.drop(columns = ["ID"])

In [4]:
columns_to_drop = dex.check_missing(df, 50,  False)
columns_to_drop

[]

In [5]:
num_cols = ["LIMIT_BAL", "AGE", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
            "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
            "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6", "default payment next month"]
df[num_cols] = df[num_cols].astype(float)

In [6]:
df_cols = df.columns.to_list()
obj_cols = dex.list_by_type(df, ['O'])
obj_cols

['SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6']

In [7]:
print(dex.list_no_variation_cols(df))
print(dex.list_contin_cols(df))
print(dex.list_by_unique(df, 2))

[]
['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
['SEX', 'default payment next month']


In [8]:
df["SEX"] = df["SEX"].apply(lambda x : "female" if x == "2" else "male")

In [9]:
X_acp = df.iloc[:, (df.columns != "default payment next month")]
y_acp = df["default payment next month"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X_acp, 
    y_acp, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_acp
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_train
)
A_train = (X_train.SEX == "female").astype(int)
A_val = (X_val.SEX == "female").astype(int)
A_test = (X_test.SEX == "female").astype(int)

## Training Basic Models

In [11]:
n_trials = 100

In [12]:
classifiers = {
    "LogisticRegression": LogisticRegression,
    "RandomForest": RandomForestClassifier,
    "LightGBM": LGBMClassifier,
    "MLPC" : MLPClassifier,
}

In [13]:
param_spaces = {
    "LogisticRegression": {
        'C': {'low': 0.001, 'high': 10, 'log': True, 'type':'float'},
        'max_iter': {'low': 1000, 'high': 1000, 'step':1, 'type':'int'},
        'penalty': {'choices': ["l1", 'l2'], 'type':'categorical'},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
        "solver" : {"choices" : ["liblinear"], "type" : "categorical"}
    },
    "RandomForestClassifier": {
        'n_estimators': {'low':10, 'high':150, 'step':20, 'type':'int'},
        'max_depth': {'low':2, 'high':10, 'type':'int'},
        'criterion': {'choices':['gini', 'entropy'], 'type':'categorical'},
        'min_samples_leaf' : {"low" : 1, "high" : 51, "step" : 5, 'type':'int'},
        "max_features" : {"low" : 0.1, "high" : 1.0, "type" : "float"},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
    },
    "LGBMClassifier": {
        'learning_rate': {'low': 0.01, 'high': 1.0, 'type': 'float', 'log': True},
        "num_leaves" : {"low" : 5, "high" : 100, "step" : 5, 'type':'int'},
        'max_depth': {'low': 2, 'high': 10, 'type': 'int'},
        'min_child_samples': {'low': 1, 'high': 51, 'step': 5, 'type': 'int'},
        'colsample_bytree': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'reg_alpha': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'reg_lambda': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'n_estimators': {'low': 10, 'high': 100, 'step': 10, 'type': 'int'},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
        "verbose" : {"choices" : [-1], 'type':'categorical'},
    },
    "MLPClassifier": {
        "hidden_layer_sizes" : {"choices" : [
            [128, 64, 32],
            [128, 64, 32, 16],
            [256, 128, 64, 32, 16],
        ], 'type':'categorical'},
        "alpha" : {'low': 0.0001, 'high': 0.01, 'type': 'float', 'log': True},
        "learning_rate" : {'choices': ['constant', 'invscaling', 'adaptive'], 'type':'categorical'},
        "learning_rate_init" : {'low': 0.001, 'high': 0.1, 'type': 'float', 'log': True},
        "early_stopping" : {'choices': [True], 'type':'categorical'},
        "max_iter" : {"choices" : [50], 'type':'categorical'},
    }
}

In [ ]:
study_logistic, model_logistic = optimize_model(LogisticRegression, param_spaces["LogisticRegression"], X_train, y_train, X_val , y_val, n_trials=n_trials)
joblib.dump(model_logistic, "models/taiwan_logistic.joblib")

In [15]:
print("Score for Logistic Regression: ", study_logistic.best_value)
print(study_logistic.best_params)

Score for Logistic Regression:  0.7743629583279172
{'C': 0.008506332302143265, 'max_iter': 1000, 'penalty': 'l2', 'class_weight': 'balanced', 'solver': 'liblinear'}


In [ ]:
study_rf, model_rf = optimize_model(RandomForestClassifier, param_spaces["RandomForestClassifier"], X_train, y_train, X_val , y_val, n_trials=n_trials)
joblib.dump(model_rf, "models/taiwan_rf.joblib")

In [27]:
print("Score for Random Forest: ", study_rf.best_value)
print(study_rf.best_params)

Score for Random Forest:  0.7854082215632396
{'n_estimators': 70, 'max_depth': 10, 'criterion': 'entropy', 'min_samples_leaf': 41, 'max_features': 0.6223181749192774, 'class_weight': 'balanced'}


In [ ]:
study_lgbm, model_lgbm = optimize_model(LGBMClassifier, param_spaces["LGBMClassifier"], X_train, y_train, X_val , y_val, n_trials=n_trials)
joblib.dump(model_lgbm, "models/taiwan_lgbm.joblib")

In [18]:
print("Score for LGBM: ", study_lgbm.best_value)
print(study_lgbm.best_params)

Score for LGBM:  0.7877014355542254
{'learning_rate': 0.05699910588443634, 'num_leaves': 45, 'max_depth': 10, 'min_child_samples': 16, 'colsample_bytree': 0.3234057672082941, 'reg_alpha': 0.45527006502059564, 'reg_lambda': 0.9669089166677883, 'n_estimators': 80, 'class_weight': 'balanced', 'verbose': -1}


In [ ]:
study_mlp, model_mlp = optimize_model(MLPClassifier, param_spaces["MLPClassifier"], X_train, y_train, X_val , y_val, n_trials=n_trials)
joblib.dump(model_mlp, "models/taiwan_mlp.joblib")

In [ ]:
print("Score for MLP: ", study_mlp.best_value)
print(study_mlp.best_params)

Score for MLP:  0.7859573737025651
{'hidden_layer_sizes': [256, 128, 64, 32, 16], 'alpha': 0.0007524655395096057, 'learning_rate': 'constant', 'learning_rate_init': 0.0025686779592416402, 'early_stopping': True, 'max_iter': 50}


## Model evaluation

In [14]:
models = {
    "LogisticRegression" : joblib.load("models/taiwan_logistic.joblib"),
    "MLP" : joblib.load("models/taiwan_mlp.joblib"),
    "RandomForest" : joblib.load("models/taiwan_rf.joblib"),
    "LightGBM" : joblib.load("models/taiwan_lgbm.joblib"),
}
ks_threshold_dict = {}
models_dict = {}
for n, m in models.items():
    ks_threshold_dict[n] = ks_threshold(y_test, m.predict_proba(X_test)[:,1])
    models_dict[n] = [
        m,
        ks_threshold_dict[n]
    ]

In [29]:
get_metrics(models_dict, X_train, y_train).round(3)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score
LogisticRegression,0.773,0.708,0.770,0.483,0.597,0.534,0.186
MLP,0.795,0.716,0.721,0.422,0.706,0.529,0.133
RandomForest,0.828,0.737,0.798,0.537,0.627,0.579,0.163
LightGBM,0.858,0.772,0.787,0.513,0.744,0.607,0.159


In [30]:
get_metrics(models_dict, X_val, y_val).round(3)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score
LogisticRegression,0.774,0.706,0.771,0.486,0.589,0.533,0.185
MLP,0.786,0.712,0.719,0.419,0.699,0.524,0.136
RandomForest,0.785,0.709,0.783,0.508,0.576,0.540,0.173
LightGBM,0.788,0.712,0.753,0.458,0.638,0.533,0.175


In [31]:
get_metrics(models_dict, X_test, y_test).round(3)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score
LogisticRegression,0.762,0.701,0.769,0.481,0.580,0.526,0.186
MLP,0.779,0.711,0.717,0.417,0.699,0.522,0.136
RandomForest,0.782,0.712,0.782,0.507,0.586,0.544,0.174
LightGBM,0.786,0.718,0.754,0.461,0.652,0.540,0.176


In [15]:
models_dict_fairness = {}
for n, m in models_dict.items():
    models_dict_fairness[n] = (m[0].predict_proba(X_test)[:,1] > m[1]).astype(int)

In [25]:
get_fairness_metrics(models_dict_fairness, y_test, X_test.SEX == "female", 0)

,DPD,EOD,AOD,APVD,GMA,balanced_accuracy
LogisticRegression,0.047421,0.038800,0.027210,0.013324,0.765189,0.700673
MLP,0.065822,0.056914,0.044550,0.013682,0.713066,0.710294
RandomForest,0.031926,0.023990,0.009129,0.018778,0.779349,0.711768
LightGBM,0.044468,0.034923,0.022814,0.017942,0.751287,0.717265


## Fairness

In [16]:
pipeline_preprocess = create_pipeline(
    X_train,
    y_train,
    lambda x : x,
)[:-1]
pipeline_preprocess.fit(X_train, y_train)
preprocess_column_names = pipeline_preprocess.get_feature_names_out()
X_train_preprocess = pipeline_preprocess.transform(X_train).toarray()
X_val_preprocess = pipeline_preprocess.transform(X_val).toarray()
X_test_preprocess = pipeline_preprocess.transform(X_test).toarray()

### Pre-processing (Reweighing / AIF360)

In [17]:
df_rw = pd.DataFrame(
    X_train_preprocess,
    columns = preprocess_column_names
)
df_rw["default payment next month"] = y_train.values
x_train_aif = BinaryLabelDataset(
    df = df_rw,
    label_names = ["default payment next month"],
    protected_attribute_names = ["cat__SEX_female"]
)
rw = Reweighing(
    unprivileged_groups = [{"cat__SEX_female": 1}],
    privileged_groups = [{"cat__SEX_female": 0}],
)
rw.fit(x_train_aif)
rw_weights = rw.transform(x_train_aif).instance_weights

In [18]:
for n in ["LogisticRegression", "RandomForest", "LightGBM"]:
    best_params = models[n]["classifier"].get_params()
    rw_model = classifiers[n](**best_params)
    rw_model.fit(X_train_preprocess, y_train, sample_weight=rw_weights)
    rw_threshold = ks_threshold(y_val, rw_model.predict_proba(X_val_preprocess)[:,1])
    models_dict_fairness[n + "Reweighing"] = rw_model.predict_proba(X_test_preprocess)[:,1] > rw_threshold

### In-processing

#### EqualOpportunityClassifier

In [19]:
gender_col_idx = np.where(preprocess_column_names == "cat__SEX_female")[0][0]
eop_class = EqualOpportunityClassifier(
    covariance_threshold = 0.1,
    sensitive_cols = gender_col_idx,
    positive_target = 1
)
eop_class.fit(X_train_preprocess, y_train)
eoq_ks_threshold = ks_threshold(y_val, eop_class.predict_proba(X_val_preprocess)[:,1])
models_dict_fairness["EqualOpportunity"] = eop_class.predict_proba(X_test_preprocess)[:,1] > eoq_ks_threshold
dop_class = DemographicParityClassifier(
    covariance_threshold = 0.1,
    sensitive_cols = gender_col_idx,
)
dop_class.fit(X_train_preprocess, y_train)
dop_ks_threshold = ks_threshold(y_val, dop_class.predict_proba(X_val_preprocess)[:,1])
models_dict_fairness["DemographicParity"] = dop_class.predict_proba(X_test_preprocess)[:,1] > dop_ks_threshold

#### FairGBM

In [20]:
fairgbm_params = models["LightGBM"]["classifier"].get_params()
del fairgbm_params["objective"]
fairgbm_params = {'learning_rate': 0.05699910588443634, 'num_leaves': 45, 'max_depth': 10, 'min_child_samples': 16, 'colsample_bytree': 0.3234057672082941, 'reg_alpha': 0.45527006502059564, 'reg_lambda': 0.9669089166677883, 'n_estimators': 80, 'verbose': -1}
fairgbm = FairGBMClassifier(
    constraint_type="FNR",
    **fairgbm_params
)
fairgbm.fit(X_train_preprocess, y_train, constraint_group=A_train)
fairgbm_threshold = ks_threshold(y_val, fairgbm.predict_proba(X_val_preprocess)[:,1])
models_dict_fairness["FairGBM"] =fairgbm.predict_proba(X_test_preprocess)[:,1] > fairgbm_threshold

### Post-processing (Threshold-Optimizer / FairLearn)

In [21]:
model_dict_thr_opt = {}
for name, model in models_dict.items():
    thr_opt = ThresholdOptimizer(
        estimator=model[0],
        constraints="equalized_odds",
        objective="balanced_accuracy_score",
        prefit=True,
        predict_method="predict_proba",
    )
    thr_opt.fit(X_train, y_train, sensitive_features=A_train)
    models_dict_fairness[name + "Thr.Opt."] = thr_opt.predict(X_test, sensitive_features=A_test)

### Fairness Evaluation

In [22]:
df_metrics_results = get_fairness_metrics(
    models_dict_fairness,
    y_test,
    A_test,
    1
)
df_metrics_results_ = df_metrics_results.copy().reset_index()

In [23]:
df_metrics_results_[["index", "AOD", "balanced_accuracy"]].round(3)

,index,AOD,balanced_accuracy
0,LogisticRegression,-0.027,0.701
1,MLP,-0.045,0.710
2,RandomForest,-0.009,0.712
3,LightGBM,-0.023,0.717
4,LogisticRegressionReweighing,0.024,0.695
5,RandomForestReweighing,-0.007,0.707
6,LightGBMReweighing,-0.015,0.710
7,EqualOpportunity,-0.029,0.701
8,DemographicParity,-0.029,0.701
9,FairGBM,0.014,0.716


In [28]:
df_metrics_results_.to_csv("results/taiwan_fairness_results.csv", index=False)

In [26]:
# Mapping each model to a unique color
color_mapping = {model: idx for idx, model in enumerate(df_metrics_results_["index"].unique())}

# Adjust fairness metrics to be "the higher the better" and scale them to be between 0 and 100
#for col in ['DPD', 'EOD', 'AOD', 'APVD']:
#    df_metrics_results_[col] = (1 - np.abs(df_metrics_results_[col])) * 100

# Scale the performance metrics to be between 0 and 100
#df_metrics_results_['GMA'] *= 100
#df_latest_adjusted['balanced_accuracy'] *= 100

# Generate the required number of colors from the Viridis colorscale
num_colors = len(color_mapping)
#colors = px.colors.qualitative.Dark2
colors = px.colors.sequential.Viridis
manual_colors = [colors[i * (len(colors) - 1) // (num_colors - 1)] for i in range(num_colors)]                                                              
# Create the Advanced Parallel Coordinates Plot with adjusted data
fig = go.Figure(data=
    go.Parcoords(
        line=dict(color=df_metrics_results_['index'].map(color_mapping), showscale=False),
        dimensions=[
            dict(label='DPD', values=df_metrics_results_['DPD'], range=[-0.1, 0.1]),
            dict(label='EOD', values=df_metrics_results_['EOD'], range=[-0.1, 0.1]),
            dict(label='AOD', values=df_metrics_results_['AOD'], range=[-0.1, 0.1]),
            dict(label='APVD', values=df_metrics_results_['APVD'], range=[-0.1, 0.1]),
            dict(label='-1 * GMA', values= -df_metrics_results_['GMA']),#, range=[0, 100]),
            dict(label='-1 * Balanced Acc.', values= - df_metrics_results_['balanced_accuracy'])#, range=[65, 100])
        ]
    )
)

# Add models to the legend using the dummy scatter plot approach and specified manual colors
for idx, (model, color) in enumerate(color_mapping.items()):
    fig.add_trace(
        go.Scatter(x=[None], y=[None],
                   mode='markers',
                   marker=dict(size=10, color=manual_colors[idx]),
                   name=model,
                   showlegend=True)
    )

# remove grid from plot
fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    font_color='black'
)

# remove xticks from plot
fig.update_xaxes(showticklabels=False)

# Update the layout and adjust the top margin
fig.update_layout(title='Taiwan Metrics German', margin=dict(t=100))

# Show the plot
fig.show()

# save fig
fig.write_image("results/taiwan_fairness_metrics.pdf")